In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
from IPython.display import clear_output

In [ ]:
!pip install gdown
!pip install transformers
!pip install sentencepiece
clear_output()

In [ ]:
!wget https://clck.ru/YT75w -O data.zip
!unzip -u data.zip

--2021-10-30 06:51:53--  https://clck.ru/YT75w
Resolving clck.ru (clck.ru)... 213.180.204.221, 2a02:6b8::221
Connecting to clck.ru (clck.ru)|213.180.204.221|:443... connected.
HTTP request sent, awaiting response... 302 FOUND
Location: https://sba.yandex.net/redirect?url=https%3A%2F%2Fstorage.yandexcloud.net%2Fhackai-datasetrsv%2FArchive.zip&client=clck&sign=b351c9bcc3485cf0572cef79be48d6df [following]
--2021-10-30 06:51:55--  https://sba.yandex.net/redirect?url=https%3A%2F%2Fstorage.yandexcloud.net%2Fhackai-datasetrsv%2FArchive.zip&client=clck&sign=b351c9bcc3485cf0572cef79be48d6df
Resolving sba.yandex.net (sba.yandex.net)... 93.158.134.232, 77.88.21.232, 213.180.204.232, ...
Connecting to sba.yandex.net (sba.yandex.net)|93.158.134.232|:443... connected.
HTTP request sent, awaiting response... 302 FOUND
Location: https://storage.yandexcloud.net/hackai-datasetrsv/Archive.zip [following]
--2021-10-30 06:51:56--  https://storage.yandexcloud.net/hackai-datasetrsv/Archive.zip
Resolving stor

In [ ]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from nltk.stem.snowball import SnowballStemmer
from tqdm.notebook import tqdm
from functools import lru_cache
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

stemmer = SnowballStemmer("russian") 

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
@lru_cache(maxsize=None)
def stem(text):
    return stemmer.stem(text)

In [ ]:
model_checkpoint = 'cointegrated/rubert-base-cased-nli-twoway'
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint)
if torch.cuda.is_available():
    model.cuda()

In [ ]:
classes = ['спортивная площадка',
            'игровая площадка',
            'лавочка',
            'освещение',
            'детский городок',
            'парк',
            'сквер',
            'место тихого отдыха',
            'аллея',
            'беседка',
            'велосипедная дорожка/велодорожка',
            'беговая дорожка',
            'зелёные насаждения',
            'асфальтовая дорожка',
            'плитка',
            'брусчатка',
            'архитектурная подсветка',
            'навигация',
            'общественный туалет',
            'пандус',
            'танцплощадка',
            'летний открытый кинотеатр']
n_classes = len(classes)

In [ ]:
def predict_zero_shot(text, label_texts, model, tokenizer,
                      label='entailment', normalize=True):
    text = " ".join([stem(i) for i in word_tokenize(text)])
    tokens = tokenizer([text] * n_classes, label_texts,
                       truncation=True, return_tensors='pt', padding=True)
    with torch.inference_mode():
        result = torch.softmax(model(**tokens.to(model.device)).logits, -1)
    proba = result[:, model.config.label2id[label]].cpu().numpy()
    if normalize:
        proba /= sum(proba)
    return label_texts[proba.argmax()]

In [ ]:
predict_zero_shot('я хочу площадку для дететей', classes, model, tokenizer)

'детский городок'

In [ ]:
df2 = pd.read_excel("/content/обращения граждан на портале народный контроль.xlsx").dropna(subset=["Текст обращения"])

In [ ]:
arr = []
for i in tqdm(df2["Текст обращения"]):
    arr.append(predict_zero_shot(i, classes, model, tokenizer))

  0%|          | 0/3137 [00:00<?, ?it/s]

In [ ]:
df2["obj_type"] = arr

In [ ]:
df2.to_csv("обращения_граждан_add_type.csv", index=False)

In [ ]:
df2.head(5)

,id,Тематика обращения,Дата создания,Категория,Муниципалитет,Текст обращения
0,3385,Пропали урны около подъездов,2021-10-02 22:03:47,Благоустройство и инфраструктура,Администрация города Чебоксары Чувашской Респу...,Летом во дворе дома Филиппа Лукина 6 проводили...
1,3383,Мусор разбросан по всему лесу (агитационные ли...,2021-10-01 19:20:12,Благоустройство и инфраструктура,Администрация города Чебоксары Чувашской Респу...,После выборов в лесу за Кадетском сквером лежа...
2,3382,Опасный перекрёсток для пешеходов и водителей,2021-10-01 13:58:08,Дороги,Администрация города Чебоксары Чувашской Респу...,Опасный перекрёсток Академика Крылова / Пирого...
3,3381,Сомнительный автомобиль в жилой зоне,2021-10-01 13:22:48,Благоустройство и инфраструктура,Администрация города Чебоксары Чувашской Респу...,"Долгое время в жилой зоне, а именно на ул. Мар..."
4,3380,321 маршрут для кого?,2021-09-29 11:25:52,Благоустройство и инфраструктура,Министерство транспорта и дорожного хозяйства ...,Водитель маршрута не остановил на остановке Чи...


In [ ]:
df.drop_duplicates(subset=[column])